In [1]:
#CSV変換等に必要なライブラリをインポート
!pip install tabula-py
!pip install pdfminer.six
!pip install PyPDF2
!pip install mojimoji
!pip install jeraconv
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.layout import LAParams
from io import StringIO
from tqdm import tqdm
from dateutil.relativedelta import relativedelta
from jeraconv import jeraconv
from PyPDF2 import PdfReader

import os
import pandas as pd
import tabula
import warnings
import mojimoji
import re

warnings.simplefilter('ignore')

In [2]:
os.getcwd()

'C:\\Users\\miyab\\OneDrive - 株式会社M＆Aバリューエンジニアリング\\共有用\\TSRデータ更新\\プログラム\\PDFからCSV'

In [3]:
#パスを指定
read_path = 'C:\\Users\\miyab\\OneDrive - 株式会社M＆Aバリューエンジニアリング\\共有用\\TSRデータ更新\\2023年上期\\2023年上期_TSR元データ\\上場'

In [4]:
#上記パスのフォルダ内を表示
dir = os.listdir(read_path)
pdf_dir = dir[0]
path = f'{read_path}\\{pdf_dir}'
dir_ = os.listdir(path)
dir_

['df_insert.csv',
 '全国上場企業_4028件.pdf',
 '全国上場企業_4028件.pdf_csv(4028)最終版.csv',
 '全国上場企業_4028件.pdf_csv.csv']

In [6]:
#上記path内のディレクトリを取得
startpdf = 0
endpdf = 1
for i in tqdm(range(startpdf,endpdf)):
    dir = os.listdir(read_path)
    dirs = []
    pdf_dir = dir[i]
    path = f'{read_path}\\{pdf_dir}'

    dir_ = os.listdir(path)
    dirs.append(dir_)
    
    pattern = '.*.pdf'
    filelist_copy = []

    for i in range(0,len(dirs)):
        for files in dirs[i]:
            if re.match(pattern,files) :
                csvfile = files
                filelist_copy.append(csvfile)
            else :
                csvfile = ''
        print(path)
        print(pdf_dir)
        print(filelist_copy)

    # "PDF"を順次読み込み
    for t in range(0,len(filelist_copy)):
#     for t in range(0,2):
        print(f'{filelist_copy[t]}を読み込み')

        os.chdir(path)
        reader = PdfReader(filelist_copy[t])
        #ページ数を取得する
        print(len(reader.pages))

        #スタートページを指定
        start = 1
        #ラストページを指定
        end = len(reader.pages)

        columns = ['企業コード(TSR)','上場区分','TSR調査年月日','企業名ﾌﾘｶﾞﾅ（半角）','代表者氏名ﾌﾘｶﾞﾅ（半角）','会社名','代表者氏名','郵便番号','所在地','電話番号','設立年月日','創業年月日','資本金（千円単位）','従業員数','業種1','業種2','業種3',
                   '営業種目','営業所・支店住所','役員','仕入先','株主構成','販売先','財務情報1_時期','財務情報1_売上(千円)','財務情報1_純利益(千円)','財務情報2_時期','財務情報2_売上(千円)','財務情報2_純利益(千円)','財務情報3_時期','財務情報3_売上(千円)','財務情報3_純利益(千円)',
                   '取引銀行','売上伸長率','利益伸長率','TSR備考','代表者住所','生年月日','出身地','出身校']
        df_ = pd.DataFrame(columns = columns)

        #"ページ"を順次読み込み
        pages = range(start,end+1)
#         pages = range(start,2)

        for page in tqdm(pages):
          try:
            df = tabula.read_pdf(filelist_copy[t],encoding = 'cp932',lattice=True, pages=page)
            value0 = df[0].columns[0] #企業コード　TSRID__c
            value1 = df[1].columns[1][2:] #上場区分 Listing_classification__c
            value2 = df[2].columns[0] #調査年月日 
            value3 = mojimoji.zen_to_han(df[3].columns[0]) #商号（カナ）KANA__C
            value4 = mojimoji.zen_to_han(df[4].columns[0]) #代表者カナ　PRESIDENTKANA__C
            value5 = df[5].columns[0] #商号（漢字）NAME
            value6 = df[6].columns[0] #代表者氏名　PRESIDENTNAME__C
            # value6_1  = value6.split(' ')[0] #姓　
            # value6_2  = value6.split(' ')[1] #名

            value7 = df[7].columns[0].strip('〒')  #郵便番号 BILLINGPOSTALCODE
            value8 = df[8].columns[1] #所在地
            value9 = df[9].columns[0] #電話番号 PHONE
            value10 = df[10].columns[1] #設立年月 STARTDATE__C
            value11 = df[11].columns[1] #創業年月
            value12 = df[12].columns[0] #資本金 CAPITAL__C
            value13 = df[13].columns[0] #従業員数

            # value14 = df[14]  #業種
            value14_1 = df[14].columns[0] #業種1
            value14_2 = df[14].iloc[0][0] #業種2
            value14_3 = df[14].iloc[1][0] #業種3

            value15 = df[15].columns[0].replace('\r','') #営業種目
            value16 = df[16].columns[1].replace('\r','') #営業所・支店
            value17 = df[17].columns[0].replace('\r','') #役員
            value18 = df[18].columns[1].replace('\r','') #仕入先
            value19 = df[19].columns[0].replace('\r','') #大株主
            value20 = df[20].columns[1].replace('\r','') #販売先

            # value21 = df[21] #業績
            value21_1_RECYM = df[21].iloc[0][0] #期間1
            value21_1_AMOUNT = df[21].iloc[0][1] #売上1
            value21_1_PROFIT = df[21].iloc[0][4] #利益1
            value21_2_RECYM = df[21].iloc[1][0] #期間2
            value21_2_AMOUNT = df[21].iloc[1][1] #売上2
            value21_2_PROFIT = df[21].iloc[1][4] #利益2
            value21_3_RECYM = df[21].iloc[2][0] #期間3
            value21_3_AMOUNT = df[21].iloc[2][1] #売上3
            value21_3_PROFIT = df[21].iloc[2][4] #利益3

            value22 = df[22].columns[1].replace('\r','') #取引銀行
            value23 = df[23].columns[1].replace('\r','') #売上伸長率
            value24 = df[24].columns[0] #利益伸長率
            value25 = df[25].columns[1].replace('\r','') #概況
          # value26 = df[26].columns[0].replace('\r','') #代表者
            value27 = df[27].columns[0].replace('\r','') #代表者住所
            value28 = df[28].columns[0].replace('\r','')[:10] #生年月日
          # value29 = df[29].columns[0].replace('\r','') #干支
            value30 = df[30].columns[0].replace('\r','')[2:] #出身地
            value31 = df[31].columns[1].replace('\r','') #出身校

            #値をリスト化
            value = [value0,value1,value2,value3,value4,value5,value6,value7,value8,value9,value10,value11,value12,value13,value14_1,value14_2,value14_3,
                    value15,value16,value17,value18,value19,value20,value21_3_RECYM,value21_3_AMOUNT,value21_3_PROFIT,value21_2_RECYM,value21_2_AMOUNT,value21_2_PROFIT,value21_1_RECYM,value21_1_AMOUNT,value21_1_PROFIT,
                    value22,value23,value24,value25,value27,value28,value30,value31]

            #テーブルを作成
            df = pd.DataFrame([value],columns = columns)
            #テーブルに追加
            df_ = df_.append(df,ignore_index = True)
            print(f'{filelist_copy[t]}を処理中({startpdf+t}/{endpdf})')
            #CSVへ変換
            try :
                df_.to_csv(f'{filelist_copy[t]}_csv.csv',encoding='cp932')
            except :
                df_.to_csv(f'{filelist_copy[t]}_csv.csv',encoding='utf-8')
            except:
            print('表示するページはありません')
#         os.chdir('C:\\Users\\ya\\OneDrive - 株式会社M＆Aバリューエンジニアリング\\共有用\\2021年下半期(50万社)\\未上場\\CSVフォルダ')
        df_.to_csv(f'{filelist_copy[t]}_csv.csv',encoding='cp932')

  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

C:\Users\miyab\OneDrive - 株式会社M＆Aバリューエンジニアリング\共有用\TSRデータ更新\2023年上期\2023年上期_TSR元データ\上場\上場
上場
['全国上場企業_4028件.pdf', '全国上場企業_4028件.pdf_csv(4028)最終版.csv', '全国上場企業_4028件.pdf_csv.csv']
全国上場企業_4028件.pdfを読み込み
4028



  0%|                                                                                            | 0/1 [00:00<?, ?it/s]Got stderr: 10月 17, 2023 3:45:42 午後 org.apache.fontbox.ttf.CmapSubtable processSubtype14
警告: Format 14 cmap table is not supported and will be ignored


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.72s/it]


全国上場企業_4028件.pdfを処理中(0/1)
全国上場企業_4028件.pdf_csv(4028)最終版.csvを読み込み


  0%|                                                                                            | 0/1 [00:03<?, ?it/s]


PdfReadError: EOF marker not found

In [ ]:
filelist_copy